In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from MembershipInferenceTest.estimators_bin_skl import ShadowModelBundle, AttackModelBundle, prepare_attack_data

In [4]:
NUM_CLASSES = 2
WIDTH = 32
HEIGHT = 32
CHANNELS = 3
SHADOW_DATASET_SIZE = 200
ATTACK_TEST_DATASET_SIZE = 200

In [5]:
target_epochs = 12
attack_epochs = 5
num_shadows = 10

In [6]:
def get_data():
    data = pd.read_csv("./data/advertising.csv")
    
    X = data.drop(['Timestamp', 'Clicked on Ad', 'Ad Topic Line', 'Country', 'City'], axis=1)
    y = data['Clicked on Ad']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

    num_columns = ['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Male']


    ct = make_column_transformer(
        (MinMaxScaler(), num_columns),
        (StandardScaler(), num_columns),
        remainder='passthrough'
    )

    X_train = ct.fit_transform(X_train)
    X_test = ct.transform(X_test)
    
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()
    
    return (X_train, y_train), (X_test, y_test)

In [7]:
def target_LG():
    """The architecture of the target (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = LogisticRegression()

    return model

In [8]:
def target_SVC():
    """The architecture of the target (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = SVC(probability=True)
    
    return model

In [9]:
def attack_model_fn():
    """Attack model that takes target model predictions and predicts membership.
    Following the original paper, this attack model is specific to the class of the input.
    AttachModelBundle creates multiple instances of this model for each class.
    """
    model = tf.keras.models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
    model.add(layers.Dense(64, activation="relu"))

    model.add(layers.Dense(1, activation="sigmoid"))
    model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [10]:
def demo():
    (X_train, y_train), (X_test, y_test) = get_data()

    # Train the target model.
    print("Training the target model...")
    target_model = target_SVC()
    target_model.fit(X_train, y_train)

    # Train the shadow models.
    smb = ShadowModelBundle(
        target_SVC,
        shadow_dataset_size=SHADOW_DATASET_SIZE,
        num_models=num_shadows,
    )

    # We assume that attacker's data were not seen in target's training.
    attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
        X_test, y_test, test_size=0.1
    )
    print(attacker_X_train.shape, attacker_X_test.shape)

    print("Training the shadow models...")
    X_shadow, y_shadow = smb.fit_transform(
        attacker_X_train,
        attacker_y_train,
        fit_kwargs=dict(
            epochs=target_epochs,
            verbose=True,
            validation_data=(attacker_X_test, attacker_y_test),
        ),
    )

    # ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
    amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

    # Fit the attack models.
    print("Training the attack models...")
    amb.fit(
        X_shadow, y_shadow, fit_kwargs=dict(epochs=attack_epochs, verbose=True)
    )

    # Test the success of the attack.

    # Prepare examples that were in the training, and out of the training.
    data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
    data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]

    # Compile them into the expected format for the AttackModelBundle.
    attack_test_data, real_membership_labels = prepare_attack_data(
        target_model, data_in, data_out
    )
    
    # Compute the attack accuracy.
    attack_guesses = amb.predict(attack_test_data)
    attack_accuracy = np.mean(attack_guesses == real_membership_labels)

    print(attack_accuracy)

In [11]:
demo()

Training the target model...
(720, 10) (80, 10)
Training the shadow models...
Training the attack models...
Epoch 1/5
63/63 [==============================] - 9s 5ms/step - loss: 0.6937 - accuracy: 0.4998
Epoch 2/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5042
Epoch 3/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.4963
Epoch 4/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6928 - accuracy: 0.5077
Epoch 5/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6934 - accuracy: 0.5146
Epoch 1/5
63/63 [==============================] - 1s 5ms/step - loss: 0.6940 - accuracy: 0.5144
Epoch 2/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6933 - accuracy: 0.5214
Epoch 3/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6927 - accuracy: 0.5038
Epoch 4/5
63/63 [==============================] - 0s 4ms/step - loss: 0.6937 - accuracy: 0.5254
Epoch 5/5
63/63 [==